In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException

import time
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import numpy as np

import requests
from bs4 import BeautifulSoup

from dbconnect import insert_articles,connect_db,insert_similar_articles

import dbconnect  # Import the module itself
import importlib

In [2]:
data = pd.read_csv('articles.csv')
simart = pd.read_csv('simart.csv')

In [8]:
print(len(data[data['Image'].isnull()])/len(data))
print()

0.16758167581675817



In [10]:
importlib.reload(dbconnect)

# Takes ~4min to upload to db
print("Attempting db connection...")
connection = dbconnect.connect_db()
print("connected to db")

# Change keywords from list to comma-separated string
data['Keywords'] = data['Keywords'].apply(lambda x: ','.join(x) if isinstance(x, list) else x)
dbconnect.insert_articles(connection, data)

print('connected, inserting...')
insert_similar_articles(connection,simart)
connection.close()
print("Inserted to db and clossed connection!")

Attempting db connection...
connected to db
connected, inserting...
Inserted to db and clossed connection!


In [6]:
def get_img(row):
    url = row['Article URL']
    source = row['Source']
    if source == 'FOX':
        print(row['Article Title'])
        # Send a GET request to the URL
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        # Find all image tags
        images = soup.find_all('img')
        
        # Loop through images and filter out ones starting with 'https://static.'
        for img in images[1:]:  # Skipping the first image (if necessary)
            image_url = img.get('src')
            if image_url and not image_url.startswith('https://static.'):
                return image_url

        # Return None if no valid image is found
        return None
    
    elif source == 'CNN':
        response = requests.get(url)
        soup = BeautifulSoup(response.content,'html.parser')

        images = soup.find_all('img')
        for img in images:
            image_url = img.get('src')
            if image_url and not 'face' in image_url and image_url.startswith('https://'):
                return image_url
        return None

In [7]:
for i,r in articles.iterrows():
    try:
        image = get_img(r)
    except:
        continue
    print(image)

Swing state community scared after Venezuelan migrant attack: lawmaker
https://a57.foxnews.com/cf-images.us-east-1.prod.boltdns.net/v1/static/694940094001/85a36e6f-65a8-4f18-a948-f53da6ba164d/5ac9a415-9a2c-453c-8166-6e846adf3c1d/1280x720/match/896/500/image.jpg?ve=1&tl=1
Blue state county bucks trend on handouts – and homeless population craters
https://a57.foxnews.com/cf-images.us-east-1.prod.boltdns.net/v1/static/694940094001/a0c283cb-4f26-4b33-9685-cdef7bd716f5/5fc0f5ac-2f07-46b5-9b5d-b02a434688de/1280x720/match/896/500/image.jpg?ve=1&tl=1
Blue state hotel near major airport hub closes its doors after half a century amid crime crisis
https://a57.foxnews.com/cf-images.us-east-1.prod.boltdns.net/v1/static/694940094001/64cbc534-2e6f-4156-8ecf-091ef559d517/f44b0ff1-524e-455d-a8f1-9b6767a3db78/1280x720/match/896/500/image.jpg?ve=1&tl=1
White House condemns Alabama shooting that police say killed 4, wounded 17 in alleged paid ‘hit’
https://a57.foxnews.com/cf-images.us-east-1.prod.boltdns.